# MERGE YOLO AND CLASSIFIER FOR SUBMISSION

## IMPORTS

In [18]:
import pandas as pd

## CONSTANTS

In [19]:
YOLO_RESULT = 'yolo.csv'
CLASSIFIER_RESULT = 'classifier.csv'

## FUNCTIONS

In [20]:
def read_pred(path):
    pred_yolo = pd.read_csv(YOLO_RESULT)
    print('YOLO: ')
    display(pred_yolo.head())
    pred_classifier = pd.read_csv(CLASSIFIER_RESULT)
    print('CLASSIFIER: ')
    display(pred_classifier.head())
    return pred_yolo, pred_classifier

In [21]:
def merge_pred(pred_yolo, pred_classifier):
    pred = pd.merge(pred_yolo, pred_classifier, on='image_id', how='left')
    display(pred.head())
    return pred

In [22]:
def filter_2cls(row):
    prob = row['target']
    if prob > 0.99:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif 0.01 <= prob <= 0.99:
        ## More change of having any diesease
        row['PredictionString'] += f' 14 {1 - prob} 0 0 1 1'
    elif 0.01 >= prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        print(prob)
        raise ValueError('Prediction must be from [0-1]')
    return row

In [23]:
def merge_2cls(pred):
    sub = pred.apply(filter_2cls, axis=1)
    display(sub.head())
    sub[['image_id', 'PredictionString']].to_csv('submission.csv', index=False)
    print('Final submission saved at submission.csv')

## RUN

#### READ PREDICTIONS

In [24]:
pred_yolo, pred_classifier = read_pred(YOLO_RESULT)

YOLO: 


,image_id,PredictionString
0,83caa8a85e03606cf57e49147d7ac569,0 0.2 1073 754 1318 976 3 0.8 785 1142 1789 1642
1,7550347fa2bb96c2354a3716dfa3a69c,3 0.0 768 1575 1908 2128 11 0.0 729 354 962 41...
2,74b23792db329cff5843e36efb8aa65a,10 0.0 139 2468 280 2648 11 0.1 166 2483 264 2645
3,94568a546be103177cb582d3e91cd2d8,11 0.1 1104 681 1324 748 11 0.1 685 712 886 77...
4,6da36354fc904b63bc03eb3884e0c35c,11 0.0 1556 608 1739 1005 11 0.0 1703 747 1821...


CLASSIFIER: 


,image_id,target
0,c9fbdd6dbbd3e5246b0184824f681e0f,0.170756
1,b12232ebf575db0226671621146f76e1,0.999787
2,e713fe6ed28f23aed83f54c56ff30189,0.995378
3,754b90e60a89d03d18dbf9b30cce6eec,0.980132
4,d7494cea30e4fa1e4b4ebd0ecc6e03e5,0.880813


#### MERGE PREDICTIONS

In [25]:
pred = merge_pred(pred_yolo, pred_classifier)

,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,0 0.2 1073 754 1318 976 3 0.8 785 1142 1789 1642,0.000371
1,7550347fa2bb96c2354a3716dfa3a69c,3 0.0 768 1575 1908 2128 11 0.0 729 354 962 41...,0.998544
2,74b23792db329cff5843e36efb8aa65a,10 0.0 139 2468 280 2648 11 0.1 166 2483 264 2645,0.999830
3,94568a546be103177cb582d3e91cd2d8,11 0.1 1104 681 1324 748 11 0.1 685 712 886 77...,0.999744
4,6da36354fc904b63bc03eb3884e0c35c,11 0.0 1556 608 1739 1005 11 0.0 1703 747 1821...,0.005593


#### CREATE SUBMISSION

In [26]:
merge_2cls(pred)

,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,0 0.2 1073 754 1318 976 3 0.8 785 1142 1789 1642,0.000371
1,7550347fa2bb96c2354a3716dfa3a69c,14 1 0 0 1 1,0.998544
2,74b23792db329cff5843e36efb8aa65a,14 1 0 0 1 1,0.999830
3,94568a546be103177cb582d3e91cd2d8,14 1 0 0 1 1,0.999744
4,6da36354fc904b63bc03eb3884e0c35c,11 0.0 1556 608 1739 1005 11 0.0 1703 747 1821...,0.005593


Final submission saved at submission.csv
